In [2]:
import os
import pyspark
from pyspark.sql import SparkSession

In [4]:
import os

os.environ["SPARK_HOME"] = "/home/ximin/anaconda3/lib/python3.12/site-packages/pyspark"
os.environ["PYSPARK_PYTHON"] = "/home/ximin/anaconda3/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "notebook"


In [5]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/05 06:48:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet

--2025-03-05 06:48:14--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.163.157.96, 3.163.157.7, 3.163.157.133, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.163.157.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308924937 (295M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2021-01.parquet.1’

fhvhv_tripdata_2021 100%[===================>] 294.61M  49.7MB/s    in 6.4s    

2025-03-05 06:48:21 (46.2 MB/s) - ‘fhvhv_tripdata_2021-01.parquet.1’ saved [308924937/308924937]



In [7]:
df = spark.read.option("header", "true").parquet('fhvhv_tripdata_2021-01.parquet')

In [8]:
# 将 DataFrame 输出为 CSV 文件，保存到当前目录
# 合并所有分区为一个，并将数据写入 temp_output 文件夹
# 直接输出到当前目录
df.write.option("header", "true").csv("output", mode="overwrite")

In [9]:
df = spark.read.option("header", "true").csv("output")

In [10]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime='2021-01-01T00:28:09.000', on_scene_datetime='2021-01-01T00:31:42.000', pickup_datetime='2021-01-01T00:33:44.000', dropoff_datetime='2021-01-01T00:49:07.000', PULocationID='230', DOLocationID='166', trip_miles='5.26', trip_time='923', base_passenger_fare='22.28', tolls='0.0', bcf='0.67', sales_tax='1.98', congestion_surcharge='2.75', airport_fee=None, tips='0.0', driver_pay='14.99', shared_request_flag='N', shared_match_flag='N', access_a_ride_flag=None, wav_request_flag='N', wav_match_flag='N'),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime='2021-01-01T00:45:56.000', on_scene_datetime='2021-01-01T00:55:19.000', pickup_datetime='2021-01-01T00:55:19.000', dropoff_datetime='2021-01-01T01:18:21.000', PULocationID='152', DOLocationID='167', trip_miles='3.65', trip_time='1382', base_passenger_fare='18.36', tolls='0.

In [11]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', StringType(), True), StructField('on_scene_datetime', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('trip_miles', StringType(), True), StructField('trip_time', StringType(), True), StructField('base_passenger_fare', StringType(), True), StructField('tolls', StringType(), True), StructField('bcf', StringType(), True), StructField('sales_tax', StringType(), True), StructField('congestion_surcharge', StringType(), True), StructField('airport_fee', StringType(), True), StructField('tips', StringType(), True), StructField('driver_pay', StringType(), True), StructField('shared_request_flag', Strin

In [12]:
df.coalesce(1).write.option("header", "true").csv("merged_output", mode="overwrite")

In [21]:
!head -n 101 fhvhv_tripdata_2021-01.csv > head.csv

head: error reading 'fhvhv_tripdata_2021-01.csv': Is a directory


In [22]:
import os

file_path = 'fhvhv_tripdata_2021-01.csv'
if os.path.isfile(file_path):
    print(f"{file_path} is a valid file.")
else:
    print(f"{file_path} is not a valid file or is a directory.")

fhvhv_tripdata_2021-01.csv is not a valid file or is a directory.


In [1]:
df.head()

NameError: name 'df' is not defined

In [14]:
df = df.repartition(24)

In [15]:
df.write.parquet('fhvhv/2021/01')

In [16]:
df = spark.read.parquet('fhvhv/2021/01')

In [18]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: string (nullable = true)
 |-- on_scene_datetime: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- trip_miles: string (nullable = true)
 |-- trip_time: string (nullable = true)
 |-- base_passenger_fare: string (nullable = true)
 |-- tolls: string (nullable = true)
 |-- bcf: string (nullable = true)
 |-- sales_tax: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)
 |-- airport_fee: string (nullable = true)
 |-- tips: string (nullable = true)
 |-- driver_pay: string (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nullable 

In [22]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID')\
  .filter(df.hvfhs_license_num == 'HV0003').take(5)
    

[Row(pickup_datetime='2021-01-01T20:18:41.000', dropoff_datetime='2021-01-01T20:26:44.000', PULocationID='258', DOLocationID='180'),
 Row(pickup_datetime='2021-01-01T00:21:15.000', dropoff_datetime='2021-01-01T00:34:18.000', PULocationID='167', DOLocationID='220'),
 Row(pickup_datetime='2021-01-01T18:44:50.000', dropoff_datetime='2021-01-01T18:50:06.000', PULocationID='232', DOLocationID='148'),
 Row(pickup_datetime='2021-01-01T21:16:05.000', dropoff_datetime='2021-01-01T21:24:26.000', PULocationID='35', DOLocationID='39'),
 Row(pickup_datetime='2021-01-02T18:50:26.000', dropoff_datetime='2021-01-02T19:04:54.000', PULocationID='151', DOLocationID='43')]

In [24]:
from pyspark.sql import functions as F

In [ ]:
F.to_date()

In [30]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [32]:
crazy_stuff_udf = F.udf(crazy_stuff)

In [34]:
df.withColumn('pickup_date', F.to_date(df.pickup_datetime))\
  .withColumn('dropoff_date', F.to_date(df.dropoff_datetime))\
  .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num))\
  .select('base_id','pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID')\
  .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/b35| 2021-01-01|  2021-01-01|         258|         180|
|  e/b30| 2021-01-01|  2021-01-01|         167|         220|
|  e/b30| 2021-01-01|  2021-01-01|         232|         148|
|  s/acd| 2021-01-01|  2021-01-01|          35|          39|
|  e/9ce| 2021-01-02|  2021-01-02|          17|          49|
|  e/b30| 2021-01-02|  2021-01-02|         151|          43|
|  e/b35| 2021-01-01|  2021-01-01|         259|         169|
|  e/9ce| 2021-01-01|  2021-01-01|          61|         188|
|  e/b30| 2021-01-01|  2021-01-01|         235|         169|
|  e/acc| 2021-01-01|  2021-01-01|          37|          36|
|  e/9ce| 2021-01-02|  2021-01-02|         121|         196|
|  e/9ce| 2021-01-01|  2021-01-01|         216|          61|
|  e/a39| 2021-01-02|  2021-01-02|           7|         263|
|  e/b3e| 2021-01-02|  2